In [26]:
import pandas as pd
import glob
import os


In [27]:

# Load all GW files
gw_folder = r"c:\Users\theoi\Documents\python stuff\FPL\FPL_dat_ pro\data\2019-20\gws"
gw_files = glob.glob(os.path.join(gw_folder, "gw*.csv"))
df_list = [pd.read_csv(f) for f in gw_files if "merged" not in f]
gws = pd.concat(df_list, ignore_index=True)

# Load fixtures and teams
fixtures = pd.read_csv(r"c:\Users\theoi\Documents\python stuff\FPL\FPL_dat_ pro\data\2019-20\fixtures.csv")
teams = pd.read_csv(r"c:\Users\theoi\Documents\python stuff\FPL\FPL_dat_ pro\data\2019-20\teams.csv")
team_id_to_name = dict(zip(teams['id'], teams['name']))

merged = pd.merge(
    gws,
    fixtures,
    left_on=['kickoff_time', 'opponent_team'],
    right_on=['kickoff_time', 'team_h'],
    how='left',
    suffixes=('', '_fixture')
)

In [28]:
fixtures.columns

Index(['code', 'event', 'finished', 'finished_provisional', 'id',
       'kickoff_time', 'minutes', 'provisional_start_time', 'started', 'stats',
       'team_a', 'team_a_difficulty', 'team_a_score', 'team_h',
       'team_h_difficulty', 'team_h_score'],
      dtype='object')

In [ ]:
mask = merged['id'].isna()
if mask.any():
    unmatched = merged[mask]
    # Only drop fixture columns that are not merge keys
    merge_keys = ['kickoff_time', 'opponent_team']
    drop_cols = [col for col in fixtures.columns if col in unmatched.columns and col not in merge_keys]
    unmatched_clean = unmatched.drop(columns=drop_cols)
    matched = pd.merge(
        unmatched_clean,
        fixtures,
        left_on=['kickoff_time', 'opponent_team'],
        right_on=['kickoff_time', 'team_a'],
        how='left',
        suffixes=('', '_fixture')
    )
    # Only assign columns that exist in both DataFrames
    assign_cols = [col for col in fixtures.columns if col in matched.columns and col in merged.columns]
    merged.loc[mask, assign_cols] = matched[assign_cols].values

# Now, infer player's team
def get_team(row):
    if row['was_home']:
        return row['team_h']
    else:
        return row['team_a']
merged['team'] = merged.apply(get_team, axis=1)
merged['team_name'] = merged['team'].map(team_id_to_name)
merged['opponent_team_name'] = merged['opponent_team'].map(team_id_to_name)


In [30]:
merged

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,stats,team_a,team_a_difficulty,team_a_score_fixture,team_h,team_h_difficulty,team_h_score_fixture,team,team_name,opponent_team_name
0,Aaron_Cresswell_376,0,0,7,0,1.5,376,8,5,0,...,"[{'h': [], 'a': [{'value': 3, 'element': 214},...",11.0,2.0,NaN,19.0,4.0,NaN,19.0,West Ham,Man City
1,Aaron_Lennon_430,0,0,3,0,0.0,430,3,0,0,...,"[{'h': [{'value': 2, 'element': 90}, {'value':...",16.0,3.0,NaN,5.0,3.0,NaN,5.0,Burnley,Southampton
2,Aaron_Mooy_516,0,0,0,0,0.0,516,7,0,0,...,"[{'h': [], 'a': [{'value': 1, 'element': 46}, ...",4.0,3.0,3.0,18.0,2.0,0.0,4.0,Brighton,Watford
3,Aaron_Ramsdale_494,0,0,11,0,0.0,494,2,1,0,...,"[{'h': [{'value': 1, 'element': 61}], 'a': [{'...",15.0,2.0,NaN,3.0,3.0,NaN,3.0,Bournemouth,Sheffield Utd
4,Aaron_Wan-Bissaka_122,0,2,34,1,16.1,122,9,0,0,...,"[{'h': [{'value': 2, 'element': 233}, {'value'...",6.0,4.0,NaN,12.0,4.0,NaN,12.0,Man Utd,Chelsea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22555,Yoshinori_Muto_260,0,0,0,0,0.0,260,83,0,0,...,"[{'h': [{'value': 1, 'element': 103}], 'a': []...",13.0,4.0,0.0,6.0,2.0,1.0,13.0,Newcastle,Chelsea
22556,Youri_Tielemans_448,0,2,30,0,37.2,448,86,1,1,...,"[{'h': [{'value': 1, 'element': 166}, {'value'...",5.0,3.0,NaN,9.0,3.0,NaN,9.0,Leicester,Burnley
22557,Yves_Bissouma_53,0,0,0,0,0.0,53,81,0,0,...,"[{'h': [{'value': 1, 'element': 29}, {'value':...",4.0,2.0,1.0,2.0,2.0,2.0,4.0,Brighton,Aston Villa
22558,Çaglar_Söyüncü_164,0,0,16,0,0.9,164,86,1,0,...,"[{'h': [{'value': 1, 'element': 166}, {'value'...",5.0,3.0,NaN,9.0,3.0,NaN,9.0,Leicester,Burnley


KeyError: 'team'